In [1]:
import os
import glob
import copy
import numpy as np
import Bio
import scipy.spatial
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from rnai_scripts import *
import bokeh.io
import bokeh.plotting

# Enable viewing Bokeh plots in the notebook
bokeh.io.output_notebook()

Loading BokehJS ...

# RNAi recoding 

## Reading in the Smed transcriptome
We read in the Smed_v6 transcriptome orfs that were extracted using orfipy. We then join them all into one string and obtain the codon frequencies. 

In [2]:
fname = 'data/dd_Smed_v6_transcripts_orfs_large3.fa' # makes smallest proteins be around 30 amino acids
descriptors, seqs = read_many_fasta(fname)
# join all ORFS into one large transcriptome 
transcriptome = ''.join(seqs)
# get aminoacidweights and codon weights 

codon_frequencies_dic = get_codon_frequencies(transcriptome) 



Now we find the GC count in the transcriptome of Sophie Walton

In [3]:
print((transcriptome.count('C') + transcriptome.count('G'))/len(transcriptome))

0.3607041500977551


I also found a published version of amino acid frequencies:

In [4]:
df = pd.read_csv('data/codon_usage_smed.csv')


AAs = df['codon'].values
freqs = df['frequency'].values/1000.

codon_frequencies_dic_published = {}
for i in range(len(AAs)):
    codon_frequencies_dic_published[AAs[i]] = freqs[i]
print(sum(freqs))

1.00000000001


Now we get frequencies of doublets

In [5]:
doubletscode = get_codon_frequencies_doublets(transcriptome)

Let's calculate the average discrepency between the doublets vs. codon frequencies. 

In [6]:
diff_published_vs_me = {}
for a in AAs:
    
    diff_published_vs_me[a] = codon_frequencies_dic_published[a] - codon_frequencies_dic[a]
values = np.array(list(diff_published_vs_me.values()))
print(np.mean(values))
print(np.mean(np.abs(values))) # values usually on order 
print(np.sum(np.abs(values)))

1.5624986385038425e-13
0.0016427577738011736
0.10513649752327511


Here we find the discrepencies between the frequencies of each doublet vs. the product frequency of the separate codons. 

In [127]:

diff_dic = {}
diff_dic_norm = {}
for pair in doubletscode.keys():
    
    # we ignore stop codons being first or conditions where ATG is at the end of a codon for now 

    if 'TAG' == pair[:3]:
        continue
    if 'TGA' == pair[:3]:
        continue
    if 'ATG' == pair[3:]:
        continue
    
    freq1 = codon_frequencies_dic[pair[:3]]
    freq2 = codon_frequencies_dic[pair[3:]]
    
    if doubletscode[pair] == 0.0:
        continue
    diff_dic_norm[pair] = (doubletscode[pair] - freq1*freq2)/(doubletscode[pair])
    diff_dic[pair] = (doubletscode[pair] - freq1*freq2)
  #  if 'TAA' == pair[:3]:
   #     print(doubletscode[pair], diff_dic_norm[pair])

In [128]:
# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Doublets - Codon1xCodon2',
    y_axis_label='ECDF',
  #  x_axis_type = 'log'
    
)
diffs, ecdf_diffs = ecdf_vals(np.array(list(diff_dic.values())))
print(np.sum(np.array(list(doubletscode.values()))))
p.circle(diffs*1e4, ecdf_diffs)

#diffs, ecdf_diffs = ecdf_vals(np.array(list(doublets.values())))
#p.circle(diffs, ecdf_diffs, color = 'orange')
bokeh.io.show(p)

1.0


In [130]:
# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='(Doublets - Codon1xCodon2)/(doubletscode[pair])',
    y_axis_label='ECDF',
  #  x_axis_type = 'log'
    
)
diffs, ecdf_diffs = ecdf_vals(np.array(list(diff_dic_norm.values())))
print(np.sum(np.array(list(doubletscode.values()))))
p.circle(diffs, ecdf_diffs)

#diffs, ecdf_diffs = ecdf_vals(np.array(list(doublets.values())))
#p.circle(diffs, ecdf_diffs, color = 'orange')
bokeh.io.show(p)

1.0


Here we look at the doublets whose normalized difference between the doublet vs the actualy codon frequency is quite small indicating that the doublet occurs much less frequent than what would be expected if the codons were independent of each other. 

In [131]:
values = np.array(list(diff_dic_norm.values()))
inds_sort = np.argsort(values)
keys = np.array(list(diff_dic_norm.keys()))
keys[inds_sort][:20]

array(['CTTAAG', 'ATCTTA', 'GTCTTA', 'CTTAGG', 'CCGCGA', 'CCTAAG',
       'CCCTAA', 'CCGCGG', 'TTTAAG', 'GCCTAA', 'TGCGAA', 'GCTTAA',
       'CTCTTA', 'GCACGG', 'TCACGT', 'ATTAAG', 'CCACGT', 'GTTAAG',
       'GGTAAG', 'ACTAAG'], dtype='<U6')

We do this for the non normalized ones to just make sure everything is not weird because of normalization.

In [132]:
values = np.array(list(diff_dic.values()))*1e4
inds_sort = np.argsort(values)
keys = np.array(list(diff_dic.keys()))
keys[inds_sort][:20]


array(['AAAAAA', 'TTTAAA', 'ATTAAA', 'ATTAAT', 'TTTAAT', 'GTTAAT',
       'CTTAAA', 'ATTAAG', 'TATAAT', 'GTTAAA', 'TTTAAG', 'TTTTTT',
       'CTTAAT', 'ATTAAC', 'AATAAG', 'ATTATA', 'TATAAA', 'ATCTTA',
       'TTCGAA', 'CAAAAA'], dtype='<U6')

We see that lots of As or Ts in a row seems to not be a good thing.... We should avoid this

We use our codon frequencies dictionary to compute CAI weights (based on the weight definition for the CAI) for all codons 

$$w_i = \frac{f_i}{\max (f_j)} i,j \in [ \text{synonymouse codons for amino acid} ]$$

Where $f_i$ is the frequency of codon $i$. 

We obtain two dictionaries: 


aminoacidweights: keys are amino acids, values are arrays of $w_i$ for all synonymous codons. The order of the codons is the as those used in aminoacidcode. 
    
gencodeweights: keys are codons, values are $w_i$ for each codon

In [13]:
aminoacidweights, gencodeweights = get_codon_weights(codon_frequencies_dic)

We pickle dump everything so we do not have to repeat the above line later. 

In [14]:
pickle.dump( aminoacidweights,
            open( "data/Smed_transcriptome_aminoacidweights.p", "wb" ) )
pickle.dump( gencodeweights, 
            open( "data/Smed_transcriptome_gencodeweights.p", "wb" ) )
pickle.dump( aminoacidcode,
            open( "data/aminoacidcode.p", "wb" ))
pickle.dump( doubletscode,
            open( "data/doubletscode.p", "wb" ))

We reload everything with pickle because why not. 

In [15]:
aminoacidweights = pickle.load( open( "data/Smed_transcriptome_aminoacidweights.p",
                                     "rb" ) )
gencodeweights = pickle.load( open( "data/Smed_transcriptome_gencodeweights.p", 
                                   "rb" ) )
aminoacidcode = pickle.load(open("data/aminoacidcode.p", 'rb'))
doubletscode = pickle.load(
            open( "data/doubletscode.p", "rb" ))

## We recode the luc ORFS!!!! 

Since SmedNluc2 is so short we must RNAi the whole thing. 

In [16]:
SmedNluc2_ORF = 'ATGGTGTTTACTTTGGAAGATTTTGTTGGAGATTGGAGACAAACTGCTGGTTACAATCTGGATCAGGTACTGGAACAAGGCGGTGTTAGTTCATTATTCCAAAACCTGGGTGTGAGTGTAACTCCGATTCAGCGAATAGTGTTGTCTGGAGAAAATGGGCTGAAGATTGATATACACGTCATAATTCCATACGAAGGCTTAAGCGGTGATCAAATGGGACAAATTGAAAAAATTTTTAAAGTAGTTTACCCAGTTGACGACCATCATTTTAAAGTTATCCTTCATTACGGTACACTGGTTATAGATGGTGTAACTCCAAATATGATCGATTATTTCGGAAGACCTTACGAAGGCATAGCCGTTTTTGATGGAAAAAAGATTACAGTAACAGGTACATTGTGGAACGGAAATAAGATTATTGACGAACGTTTAATTAACCCAGATGGAAGTTTGCTCTTTAGAGTTACAATTAATGGTGTGACAGGATGGAGATTATGCGAACGGATACTCGCGTAA'

In [17]:
SmedNluc2_protein = 'MVFTLEDFVGDWRQTAGYNLDQVLEQGGVSSLFQNLGVSVTPIQRIVLSGENGLKIDIHVIIPYEGLSGDQMGQIEKIFKVVYPVDDHHFKVILHYGTLVIDGVTPNMIDYFGRPYEGIAVFDGKKITVTGTLWNGNKIIDERLINPDGSLLFRVTINGVTGWRLCERILA*'

In [18]:
Hluc_ORF = 'ATGGTCTTCACACTCGAAGATTTCGTTGGGGACTGGCGACAGACAGCCGGCTACAACCTGGACCAAGTCCTTGAACAGGGAGGTGTGTCCAGTTTGTTTCAGAATCTCGGGGTGTCCGTAACTCCGATCCAAAGGATTGTCCTGAGCGGTGAAAATGGGCTGAAGATCGACATCCATGTCATCATCCCGTATGAAGGTCTGAGCGGCGACCAAATGGGCCAGATCGAAAAAATTTTTAAGGTGGTGTACCCTGTGGATGATCATCACTTTAAGGTGATCCTGCACTATGGCACACTGGTAATCGACGGGGTTACGCCGAACATGATCGACTATTTCGGACGGCCGTATGAAGGCATCGCCGTGTTCGACGGCAAAAAGATCACTGTAACAGGGACCCTGTGGAACGGCAACAAAATTATCGACGAGCGCCTGATCAACCCCGACGGCTCCCTGCTGTTCCGAGTAACCATCAACGGAGTGACCGGCTGGCGGCTGTGCGAACGCATTCTGGCGTAA'

I wonder what the CAI for each ORF is?

In [19]:
print('CAI for SMed Nuc:', get_CAI(SmedNluc2_ORF, gencodeweights))
print('CAI for Human Nuc:', get_CAI(Hluc_ORF, gencodeweights))
print('Hamming Distance vs Smed vs Human Nuc', get_hamming_dist(SmedNluc2_ORF, Hluc_ORF))

CAI for SMed Nuc: 0.737701636271008
CAI for Human Nuc: 0.5470635190087074
Hamming Distance vs Smed vs Human Nuc 0.25775193798449614


Now we can use the function get_RNAi_seq to randomly sample different recoded Luc proteins. 

The function get_RNAi_seq requires the ORF, protein sequence, an aminoacidweights and gencodeweights dictionary. We run 1000 random samples and do not enforce that every codon be different. It returns the list of tested sequences (seqs), scores ($CAI + D$/2) for each sequence, codon adaptation indices (CAIs), and Hamming distances (dists = $D$). 

In [20]:
opposite_codon_frequencies = get_opposite_codon_frequencies(codon_frequencies_dic)
# note this will not sum to 1 goal is to give lower higher weight etc. 

In [138]:
opposite_aminoacidweights, opposite_gencodeweights = \
get_codon_weights(opposite_codon_frequencies)

In [140]:
seqs, scores, cais, dists = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights,    gencodeweights,
                                         pairs = False,
                          trials = 1000,  enforce_different_codons = False, random = True)


best_seq, best_score, best_cai, best_dist = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1,  enforce_different_codons = False, random = False)

best_doublet = get_doublest_likelihood(best_seq[0], doubletscode)
doublets_scores = np.array([get_doublest_likelihood(seq, doubletscode) for seq in seqs])
print(best_cai, best_dist, best_doublet)

[1.0] [0.187984496124031] -1253.807096386507


We fix the above sequence to avoid repetative elements. 

In [141]:
seqs_opposite, _, cais_opposite, dists_opposite = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, opposite_aminoacidweights, 
                            opposite_gencodeweights, trials = 1000,  enforce_different_codons = False, random = True)


best_seq_opposite, _,best_cai_opposite, best_dist_opposite = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, opposite_aminoacidweights, 
                            opposite_gencodeweights, trials = 1,  enforce_different_codons = False, random = False)

best_doublet_opposite = get_doublest_likelihood(best_seq_opposite[0], doubletscode)
#doublets_scores = np.array([get_doublest_likelihood(seq, doubletscode) for seq in seqs])
print(best_cai_opposite, best_dist_opposite, best_doublet_opposite)

[1.0] [0.2926356589147287] -1569.0248042766045


We redo the process but enforce that every codon must be different. 

In [142]:
seqs_diff, scores_diff, cais_diff, dists_diff = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights,
                            gencodeweights, trials = 1000,  enforce_different_codons = True, random = True)

best_seq_diff, best_score_diff, best_cai_diff, best_dist_diff = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1,  enforce_different_codons = True, random = False)
best_doublet_diff = get_doublest_likelihood(best_seq_diff[0], doubletscode)
doublets_scores_diff = np.array([get_doublest_likelihood(seq, doubletscode) for seq in seqs_diff])
print(best_cai_diff, best_dist_diff, best_doublet_diff)

[0.7226465538224905] [0.35658914728682173] -1367.970980554914


In [ ]:
seqs_diff_nW, scores_diff, cais_diff_nW, dists_diff_nW = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1000,  enforce_different_codons = True, random = True, no_wobble = True,)

best_seq_diff, best_score_diff, best_cai_diff, best_dist_diff = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1,  enforce_different_codons = True, random = False,  no_wobble = True,)
best_doublet_diff = get_doublest_likelihood(best_seq_diff[0], doubletscode)
doublets_scores_diff_nW = np.array([get_doublest_likelihood(seq, doubletscode) for seq in seqs_diff])
print(best_cai_diff, best_dist_diff, best_doublet_diff)

We define a function to compute ECDFs

In [104]:
def sliding_window_RNAi_recoding(recode, ORF, protein, aminoacidweights, gencodeweights, 
                               random = False, no_wobble = False, enforce_different_codons = True, wiggle = False):
    assert(recode % 3 == 0.)
        
    n_recodes = (len(ORF) - recode)//3
    recode = recode // 3
    cais = np.zeros(n_recodes)
    dists = np.zeros(n_recodes)
    cais_full = np.zeros(n_recodes)
    dists_full = np.zeros(n_recodes)
    seqs = []
    seqs_small = []
    for i in range(n_recodes):
        ORF_small = ORF[i*3:3*(i + recode)]
       # print(len(ORF_small))
        protein_small = protein[i:i + recode]
        seq, _, cai, dist = get_RNAi_seq(ORF_small, protein_small, aminoacidweights, gencodeweights, trials = 1, 
                 random = random,
                 no_wobble = no_wobble,
                 enforce_different_codons = enforce_different_codons, wiggle = wiggle)
        
        
        ORF_recode = ORF[:i*3] + seq[0] + ORF[3*(i + recode):]
        seqs_small.append(seq)
        #print(len(ORF_recode))
        cai_full = get_CAI(ORF_recode, gencodeweights)
        dist_full = get_hamming_dist(ORF_recode, ORF)
        
        cais_full[i] = cai_full
        dists_full[i] = dist_full
        seqs.append(seq[0])
        cais[i] = cai[0]
        dists[i] = dist[0]
    return seqs, seqs_small, cais, dists, cais_full, dists_full


In [88]:
cais_full

array([0.71912367, 0.72295418, 0.71501148, 0.7215922 , 0.72072626,
       0.71830039, 0.727316  , 0.72933477, 0.72799105, 0.72879636,
       0.73253619, 0.73667885, 0.74126413, 0.74070984, 0.74222336,
       0.73902591, 0.73825175, 0.73428883, 0.72629216, 0.73212022,
       0.73134691, 0.73460023, 0.72940355, 0.72689633, 0.72110005,
       0.72579644, 0.73118066, 0.7223809 , 0.72589468, 0.73061963,
       0.73312574, 0.73842544])

In [90]:
dists

array([0.35      , 0.35238095, 0.35238095, 0.3547619 , 0.3547619 ,
       0.3547619 , 0.3547619 , 0.3547619 , 0.3547619 , 0.3547619 ,
       0.35952381, 0.35952381, 0.36428571, 0.36428571, 0.36428571,
       0.36428571, 0.36428571, 0.36428571, 0.36428571, 0.36428571,
       0.36190476, 0.36190476, 0.36190476, 0.35952381, 0.35714286,
       0.35714286, 0.35714286, 0.35714286, 0.35952381, 0.35952381,
       0.35714286, 0.35714286])

In [105]:
recode = 150

seqs, seqs_small, cais, dists, cais_full, dists_full = sliding_window_RNAi_recoding(recode, SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, gencodeweights, 
                                                 random = False, no_wobble = True, enforce_different_codons = True, wiggle = False)

In [107]:
best_dist_ind = np.argmax(dists)
best_dist_ind
seqs_small[best_dist_ind]

['AACTTAGACCAAGTTTTAGAGCAGGGAGGAGTATCATCTCTTTTTCAGAATTTAGGAGTTTCAGTTACACCAATACAAAGAATTGTTTTATCAGGTGAGAACGGATTAAAAATAGACATTCATGTAATTATACCTTATGAGGGACTTTCA']

In [ ]:
codons = len(seq//3)
for c in range(codons):
    codon_pair = seq[3*c:3*c + 6]
    if diff_dic[codon_pair] < -1e-4
    

In [119]:
good = seqs_small[best_dist_ind][0]
for c in range(50):
    start = 3*c
    codon_pair = good[3*c:3*c + 6]
    if diff_dic[codon_pair] < -1e-4:
        print(codon_pair, diff_dic[codon_pair] )
        alternative_codons = 


AACTTA -0.00019597497481991995
TTAGGA -0.00010845825389880847
ATACAA -0.00019290093742329976
TCAGGT -0.00011100149388862518
ATTATA -0.0003084165644646487


KeyError: 'TCA'

In [122]:
diff_dic['AAAAAA']
gencode['ATT']
aminoacidcode['I']

['ATA', 'ATC', 'ATT']

In [125]:
diff_dic['ATCATA']

-4.4029647474448414e-05

We plot ECDFs of the CAIs.  

In [33]:
# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='CAI',
    y_axis_label='ECDF',
   # x_range = (-1,1)
    
)
cais, ecdf_cais = ecdf_vals(cais)
p.circle(cais, ecdf_cais, legend_label = 'CAI')

cais_diff, ecdf_cais_diff = ecdf_vals(cais_diff)
p.circle(cais_diff, ecdf_cais_diff, legend_label = 'CAI, All Different', color = 'orange')



cais_nW, ecdf_cais_nW = ecdf_vals(cais_diff_nW)
p.circle(cais_nW, ecdf_cais_nW, legend_label = 'CAI, All Different, Wiggle', color = 'green')


#p.legend.location = 'bottom_left'
p.legend.visible = False
#p.add_layout(legend, 'right')
bokeh.io.show(p)

We plot ECDFs of the hamming distances 

In [34]:
# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Hamming Distance',
    y_axis_label='ECDF',
    
)
dists, ecdf_dists = ecdf_vals(dists)
p.circle(dists, ecdf_dists, legend_label = 'Not all different ')

dists_diff, ecdf_dists_diff = ecdf_vals(dists_diff)
p.circle(dists_diff, ecdf_dists_diff, legend_label = 'all different', color = 'orange')


dists_diff_wiggle, ecdf_dists_diff_wiggle = ecdf_vals(dists_diff_nW)
p.circle(dists_diff_wiggle, ecdf_dists_diff_wiggle, legend_label = 'wiggle', color = 'green')



p.legend.visible = False
#p.x_range = bokeh.models.Range1d(.1, .6)
bokeh.io.show(p)

In [ ]:
# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Log Doublets Likelihood',
    y_axis_label='ECDF',
    
)
dists, ecdf_dists = ecdf_vals(doublets_scores)
p.circle(dists, ecdf_dists, legend_label = 'Not all different ')

dists_diff, ecdf_dists_diff = ecdf_vals(doublets_scores_diff)
p.circle(dists_diff, ecdf_dists_diff, legend_label = 'all different', color = 'orange')


dists_diff_wiggle, ecdf_dists_diff_wiggle = ecdf_vals(doublets_scores_wiggle)
p.circle(dists_diff_wiggle, ecdf_dists_diff_wiggle, legend_label = 'wiggle', color = 'green')



#p.legend.location = 'bottom_right'
p.legend.visible = False
bokeh.io.show(p)